# Evolution Game
- Created and modified by John Tan Chong Min on 18 Aug 2023
- Check out the pricing of OpenAI text and image generation here: https://openai.com/pricing
- Get your API key here: https://platform.openai.com/account/api-keys

## Steps
- The game consists of three phases, which repeats continuously until game over: 
    - Choose an attribute: Conditioned on the story, player will choose one attribute to add based on a list of three randomly generated attributes and description of attributes by you. 
    - Fight a creature: Player will choose one creature to fight based on a list of three randomly generated creatures by you. Be realistic. 
    - Evolve: Player will choose between three species to evolve to, generated by you. Habitat will change based on the new species. This should be the next class in the taxonomy tree, for example, bacteria to protozoa. Once evolved, the species will change but the attributes will remain. Next phase will be choose an attribute.

- Uses a rule-based phase tracker to see what phase it is
- Uses memory of the previous phase to guide generation of the next phase
- Gets GPT to generate information based on the phase using Chain of Thought prompting, starting from Species, Attribute, Habitat, Phase to generate plausible Options
- Uses DALL-E to generate plausible images based on the Option Description

In [1]:
import os
import openai
import json
import re
from IPython.display import display, HTML

#API Keys
os.environ['OPENAI_API_TOKEN'] = 'YOUR_API_KEY_HERE'
openai.api_key = os.environ['OPENAI_API_TOKEN']

In [37]:
def strict_output(system_prompt, user_prompt, output_format, default_category = "", output_value_only = False,
                  model = 'gpt-3.5-turbo', temperature = 0, num_tries = 2, verbose = False):
    ''' Ensures that OpenAI will always adhere to the desired output json format. 
    Uses rule-based iterative feedback to ask GPT to self-correct.
    Keeps trying up to num_tries it it does not. Returns empty json if unable to after num_tries iterations.
    If output field is a list, will treat as a classification problem and output best classification category.
    Text enclosed within < > will generated by GPT accordingly'''

    # if the user input is in a list, we also process the output as a list of json
    list_input = isinstance(user_prompt, list)
    # if the output format contains dynamic elements of < or >, then add to the prompt to handle dynamic elements
    dynamic_elements = '<' in str(output_format)
    # if the output format contains list elements of [ or ], then we add to the prompt to handle lists
    list_output = '[' in str(output_format)
    
    # start off with no error message
    error_msg = ''
    
    for i in range(num_tries):
        
        output_format_prompt = f'''\nYou are to output the following in json format: {output_format}. 
Do not put quotation marks ' or " or escape character \ when describing attributes.'''
        
        if list_output:
            output_format_prompt += f'''\nIf output field is a list, classify output into the best element of the list.'''
        
        # if output_format contains dynamic elements, process it accordingly
        if dynamic_elements: 
            output_format_prompt += f'''
Any text enclosed by < and > indicates you must generate content to replace it. Example input: Go to <location>, Example output: Go to the garden
Any output key containing < and > indicates you must generate the key name to replace it. Example input: {{'<location>': 'description of location'}}, Example output: {{'school': 'a place for education'}}'''

        # if input is in a list format, ask it to generate json in a list
        if list_input:
            output_format_prompt += '''\nGenerate a list of json, one json for each input element.'''
            
        # Use OpenAI to get a response
        response = openai.ChatCompletion.create(
          temperature = temperature,
          model=model,
          messages=[
            {"role": "system", "content": system_prompt + output_format_prompt + error_msg},
            {"role": "user", "content": str(user_prompt)}
          ]
        )

        res = response['choices'][0]['message']['content'].replace('\'', '"')
        
        # ensure that we don't replace away aprostophes in text 
        res = re.sub(r"(\w)\"(\w)", r"\1'\2", res)

        if verbose:
            print('System prompt:', system_prompt + output_format_prompt + error_msg)
            print('\nUser prompt:', str(user_prompt))
            print('\nGPT response:', res)
        
        # try-catch block to ensure output format is adhered to
        try:
            output = json.loads(res)
            if isinstance(user_prompt, list):
                if not isinstance(output, list): raise Exception("Output format not in a list of json.")
            else:
                output = [output]
                
            # check for each element in the output_list, the format is correctly adhered to
            for index in range(len(output)):
                for key in output_format.keys():
                    # unable to ensure accuracy of dynamic output header, so skip it
                    if '<' in key or '>' in key: continue
                    # if output field missing, raise an error
                    if key not in output[index]: raise Exception(f"{key} not in json output")
                    # check that one of the choices given for the list of words is an unknown
                    if isinstance(output_format[key], list):
                        choices = output_format[key]
                        # ensure output is not a list
                        if isinstance(output[index][key], list):
                            output[index][key] = output[index][key][0]
                        # output the default category (if any) if GPT is unable to identify the category
                        if output[index][key] not in choices and default_category:
                            output[index][key] = default_category
                        # if the output is a description format, get only the label
                        if ':' in output[index][key]:
                            output[index][key] = output[index][key].split(':')[0]
                            
                # if we just want the values for the outputs
                if output_value_only:
                    output[index] = [value for value in output[index].values()]
                    # just output without the list if there is only one element
                    if len(output[index]) == 1:
                        output[index] = output[index][0]
                    
            return output if list_input else output[0]

        except Exception as e:
            error_msg = f"\n\nResult: {res}\n\nError message: {str(e)}"
            print("An exception occurred:", str(e))
            print("Current invalid json format:", res)
         
    return {}

In [38]:
def display_images(option1, option2, option3):
    defaulturl = "https://th.bing.com/th/id/OIG.lVXjWwlHyIo4QdjnC1YE?pid=ImgGn"
    try:
        response = openai.Image.create(
          prompt=option1,
          n=1,
          size="1024x1024"
        )
        image_url_1 = response['data'][0]['url']
    except Exception as e:
        image_url_1 = defaulturl
    
    try:
        response = openai.Image.create(
          prompt=option2,
          n=1,
          size="1024x1024"
        )
        image_url_2 = response['data'][0]['url']
    except Exception as e:
        image_url_2 = defaulturl
        
    try:
        response = openai.Image.create(
          prompt=option3,
          n=1,
          size="1024x1024"
        )
        image_url_3 = response['data'][0]['url']
    except Exception as e:
        image_url_3 = defaulturl
    
    return [image_url_1, image_url_2, image_url_3]

# Main Game

In [53]:
memory = {'Phase': 'Evolve to a new species', 'Habitat': 'Aquatic Pond', 'Species': 'Bacteria', 'Attribute': 'Resilient',
'Option 1': 'Protozoa'}
player_choice = '1'

counter = 0
while True:
    phase = ['Choose an Attribute', 'Fight a Creature', 'Evolve to a new species'][counter%3]
    res = strict_output(system_prompt = f'''You are the game host for the ”Evolution" game.

The game consists of three phases, which repeats continuously until game over: 
- Choose an attribute: Conditioned on the story, player will choose one attribute to add based on a list of three randomly generated attributes and description of attributes by you. 

- Fight a creature: Player will choose one creature to fight based on a list of three randomly generated creatures by you. Be realistic. 

– Evolve: Player will choose between three species to evolve to, generated by you. Habitat will change based on the new species. This should be the next class in the taxonomy tree, for example, bacteria to protozoa. Once evolved, the species will change but the attributes will remain. Next phase will be choose an attribute.

Previous Phase: {memory}

Current Phase: {phase}''',

user_prompt = f'''Player Choice for Previous Phase: {player_choice}''',
output_format = {"Outcome": "<Vivid description of outcome of previous phase based on Player Choice>",
                 "Species": "Type of Species of Player", "Attribute": "Attributes in a list", 
                  "Habitat": "Current Habitat", f"{phase}": "<Description of phase>",
                  "Option 1": f"<Name> - <Description based on {phase}>",
                  "Option 2": f"<Name> - <Description based on {phase}>",
                  "Option 3": f"<Name> - <Description based on {phase}>"},
temperature = 0.8)

    # if output is not generated, repeat generation
    ungenerated = False
    for each in res:
        if "<" in each:
            ungenerated = True
    if ungenerated:
        continue
        
    ## TODO: Make the UI prettier by using Gradio
    memory = res
    for key in res:
        print(f'{key}: {res[key]}')
        
    # remove outcome to prevent conditioning on it the next phase
    del memory['Outcome']
        
    image_urls = display_images(res["Option 1"], res["Option 2"], res["Option 3"])
    
    # HTML and CSS for displaying smaller images side by side
    html = '<div style="display:flex; justify-content:center;">'
    for url in image_urls:
        html += f'<img src="{url}" style="width:200px; height:150px; margin-right: 10px;">'
    html += '</div>'

    # Display the HTML
    display(HTML(html))
        
    player_choice = input('Enter your choice\n')
    while player_choice not in ['1', '2', '3']:
         player_choice = input('Invalid choice. Enter your choice (1, 2 or 3).\n')
            
    counter += 1
    print()

Outcome: Congratulations! You have successfully evolved from a Bacteria to a Protozoa. With your new species, you have gained the ability to move using cilia and flagella, allowing you to explore your aquatic pond habitat more effectively.
Species: Protozoa
Attribute: Resilient
Habitat: Aquatic Pond
Choose an Attribute: Choose an attribute to further enhance your Protozoa species:
Option 1: Predatory - Develop specialized structures to capture and consume other microorganisms in the pond.
Option 2: Photosynthetic - Gain the ability to harness sunlight for energy production, reducing your dependence on organic matter.
Option 3: Multi-flagellated - Grow additional flagella to increase your swimming speed and agility in the water.


Enter your choice
 1



Outcome: Your Protozoa species has developed specialized structures to capture and consume other microorganisms in the pond. With this predatory advantage, your species has become a formidable predator in the ecosystem, hunting down and feeding on smaller organisms in the aquatic pond.
Species: Protozoa
Attribute: ['Resilient', 'Predatory']
Habitat: Aquatic Pond
Fight a Creature: Choose a creature to fight:
Option 1: Amoeba - A single-celled organism with a shapeless body, capable of engulfing and digesting other microorganisms.
Option 2: Paramecium - A unicellular organism with a slipper-like shape and cilia for movement and feeding.
Option 3: Euglena - A flagellated organism with a long whip-like tail, capable of both photosynthesis and predation.


Enter your choice
 1



Outcome: The amoeba engulfed the prey, digesting it and absorbing its nutrients. The amoeba successfully defended itself and satisfied its hunger.
Species: Amoeba
Attribute: ['Resilient', 'Predatory']
Habitat: Aquatic Pond
Evolve to a new species: Choose a new species to evolve to:
Option 1: Parasitic Worm - A multicellular organism that feeds on the internal or external surface of a host organism, causing harm or disease.
Option 2: Freshwater Sponge - A filter-feeding organism that extracts nutrients from the water by pumping it through its porous body structure.
Option 3: Jellyfish - A gelatinous organism with tentacles equipped with stinging cells, capable of capturing and paralyzing prey.


...

Enter your choice
 2



Outcome: The Giant Water Bug put up a fierce fight with its powerful bite and camouflage abilities, but the Burrowing Beetle's thick exoskeleton provided excellent protection. After a long and intense battle, the Burrowing Beetle emerged victorious.
Species: Burrowing Beetle
Attribute: ['Resilient', 'Predatory', 'Segmented Body', 'Thick Exoskeleton']
Habitat: Aquatic Pond
Evolve to a new species: Evolve to a new species:
Option 1: Water Scorpion - A predatory insect with a long tail-like structure and a powerful grasping claw, capable of catching small fish and insects in the water.
Option 2: Dragonfly - A fast and agile insect with large wings and a strong jaw, known for its impressive hunting skills and ability to catch prey mid-air.
Option 3: Diving Beetle - An aquatic beetle with specialized legs for swimming and diving, equipped with strong mandibles to catch and consume underwater prey.


Enter your choice
 2



Outcome: You have evolved into a Dragonfly! With your impressive hunting skills and ability to catch prey mid-air, you soar through the skies with unmatched agility. Your segmented body and thick exoskeleton remain, providing you with a sturdy defense against predators. Your new species brings you to a new habitat: the open fields and forests, where you can thrive and explore the vast landscapes.
Species: Dragonfly
Attribute: ['Resilient', 'Predatory', 'Segmented Body', 'Thick Exoskeleton']
Habitat: Open Fields and Forests
Choose an Attribute: Choose an Attribute:
Option 1: Sharp Vision - Your compound eyes have the ability to detect even the slightest movement, allowing you to spot prey from great distances.
Option 2: Camouflage - With your ability to change color and blend into your surroundings, you become almost invisible to predators and unsuspecting prey.
Option 3: Wingspan Increase - Your wings grow larger, granting you increased speed and maneuverability in the air.


KeyboardInterrupt: Interrupted by user